In [1]:
# required modules
import pandas as pd
import numpy as np 
import matplotlib
%matplotlib inline
matplotlib.style.use('default')
import requests
import json

In [9]:
# data and google api key setup
#google_api_access_key = open('../.env/.google_api_access_key').read()  #burak's key
google_api_access_key = open('../.env/.key').read()                     #Tchucky's key

# Google Places Nearby Search

In [ ]:
# Sample coordinate
lat, lng = grid_points_array[1][0], grid_points_array[1][1]
print(lat, lng)

# Sample response / Google nearby search
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

params = {
    'keyword':'restaurant',
    'location':f'{lat},{lng}',
    'radius':100,
    'region':'ch',
    'key':google_api_access_key
}

response_sample = requests.get(base_url, params=params).json()
df_sample = pd.json_normalize(response_sample['results'])\
        [['name',
          'place_id',
          'types',
          'price_level',
          'rating',
          'user_ratings_total',
          'vicinity',
          'plus_code.global_code',
          'geometry.location.lat',
          'geometry.location.lng']]
df_sample

In [ ]:
%%time
#this is my test to check the git (Tchucky)
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?' # base url

df_restaurants = pd.DataFrame()

for row in grid_points_array:
    lat, lng = row[0], row[1]
    radius = 100 #meters

    params = {
        'keyword':'restaurant',
        'location':f'{lat},{lng}',
        'radius':radius,
        'region':'ch',
        'key':google_api_access_key
    }
    
    try:
        response_basic = requests.get(base_url, params=params).json()

        # create a temporarily dataframe to hold response
        df_temp= pd.json_normalize(response_basic['results'])\
            [['name',
              'place_id',
              'types',
              'price_level',
              'rating',
              'user_ratings_total',
              'vicinity',
              'plus_code.global_code',
              'geometry.location.lat',
              'geometry.location.lng']]

        # concat and remove duplicates (beware drop_duplicates do not work due to lists values)
        drop_ref = df_temp.drop(columns=['types']).columns.to_list()
        df_restaurants = pd.concat([df_restaurants, df_temp], axis=0)\
            .drop_duplicates(drop_ref)\
            .reset_index(drop=True)
        
    except:
        continue
print(f'last_row: {row}')

In [ ]:
df_restaurants

In [ ]:
df_restaurants.to_csv('../raw_data/raw_data_google_api.csv', encoding='utf-8-sig')

# Google Places Details Search

In [47]:
df_id = pd.read_csv('../raw_data/raw_data_google_api.csv', encoding='utf-8-sig')['place_id']
# df_id = df_id.head(5)
# df_id

In [ ]:
# Sample response / Google Places Details search
base_url = 'https://maps.googleapis.com/maps/api/place/details/json?'
fields = 'place_id,name,international_phone_number,formatted_address,types,rating,review,user_ratings_total,price_level'

#grab the first line, just to have the columns in the daframe for pandas
params = {
    'place_id':df_id[0],
    'fields':fields,
    'key':google_api_access_key}
response_details_sample = requests.get(base_url, params=params, timeout=2).json()
df_details_sample = pd.json_normalize(response_details_sample['result'])
c=1

for i in df_id:
    # field dimension to investigate
    print('\r\r\r\r',end=str(c),flush=True)
    c+=1
    params = {
        'place_id':i,
        'fields':fields,
        'key':google_api_access_key
    }
    response_details_sample = requests.get(base_url, params=params, timeout=5).json()
    try:
        df_details_sample = pd.concat([df_details_sample,pd.json_normalize(response_details_sample['result'])])
    except:
        continue

5

In [64]:
df_details_sample.to_csv('../raw_data/raw_data_google_details.csv', encoding='utf-8-sig')

In [63]:
df_details_sample.shape

(115, 9)

In [59]:
for i in range(50):
    print('\r',end=str(i))
    #print('test')

49